In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
class Model(nn.Module):
    def __init__(self, in_features=4, h1=8, h2=9, out_features=3):
        super().__init__()
        self.fc1 = nn.Linear(in_features, h1)
        self.fc2 = nn.Linear(h1, h2)
        self.out = nn.Linear(h2, out_features)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)
        return x

In [6]:
torch.manual_seed(32)
model = Model()

In [7]:
next(model.parameters()).is_cuda

False

In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

gpumodel = model.to(device)

In [9]:
next(gpumodel.parameters()).is_cuda

True

In [10]:
df = pd.read_csv('../Data/IRIS/iris.csv')

In [11]:
df

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,Iris-virginica
146,147,6.3,2.5,5.0,1.9,Iris-virginica
147,148,6.5,3.0,5.2,2.0,Iris-virginica
148,149,6.2,3.4,5.4,2.3,Iris-virginica


In [12]:
df.loc[df['Species'] == "Iris-setosa"] = 0
df.loc[df['Species'] == "Iris-versicolor"] = 1
df.loc[df['Species'] == "Iris-virginica"] = 2

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             150 non-null    int64  
 1   SepalLengthCm  150 non-null    float64
 2   SepalWidthCm   150 non-null    float64
 3   PetalLengthCm  150 non-null    float64
 4   PetalWidthCm   150 non-null    float64
 5   Species        150 non-null    object 
dtypes: float64(4), int64(1), object(1)
memory usage: 7.2+ KB


In [14]:
df['Species'] = df['Species'].astype(float)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             150 non-null    int64  
 1   SepalLengthCm  150 non-null    float64
 2   SepalWidthCm   150 non-null    float64
 3   PetalLengthCm  150 non-null    float64
 4   PetalWidthCm   150 non-null    float64
 5   Species        150 non-null    float64
dtypes: float64(5), int64(1)
memory usage: 7.2 KB


In [15]:
df

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,0,0.0,0.0,0.0,0.0,0.0
1,0,0.0,0.0,0.0,0.0,0.0
2,0,0.0,0.0,0.0,0.0,0.0
3,0,0.0,0.0,0.0,0.0,0.0
4,0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
145,2,2.0,2.0,2.0,2.0,2.0
146,2,2.0,2.0,2.0,2.0,2.0
147,2,2.0,2.0,2.0,2.0,2.0
148,2,2.0,2.0,2.0,2.0,2.0


In [16]:
df.drop(['Id'],axis=1,inplace=True)

In [17]:
df

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...
145,2.0,2.0,2.0,2.0,2.0
146,2.0,2.0,2.0,2.0,2.0
147,2.0,2.0,2.0,2.0,2.0
148,2.0,2.0,2.0,2.0,2.0


In [18]:
X = df.drop('Species', axis=1).values

In [19]:
y = df['Species'].values

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=33)

In [21]:
X_train = torch.FloatTensor(X_train).to(device)
X_test = torch.FloatTensor(X_test).to(device)
y_train = torch.LongTensor(y_train).to(device)
y_test = torch.LongTensor(y_test).to(device)

In [22]:
trainloader = DataLoader(X_train, batch_size=60, shuffle=True, pin_memory=True)
testloader = DataLoader(X_test, batch_size=60, shuffle=False, pin_memory=True)

In [23]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [24]:
import time

start = time.time()
epochs = 100
losses = []

for i in range(epochs):
    y_pred = gpumodel.forward(X_train)
    y_pred = y_pred.type(torch.FloatTensor).to(device)
    loss = criterion(y_pred, y_train)
    losses.append(loss)

    if i%10 == 0:
        print(f'Epoch: {i} loss {loss.item()}')

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

end = time.time()-start
print(f'Total Time: {end}')

Epoch: 0 loss 1.1012368202209473
Epoch: 10 loss 0.9528831839561462
Epoch: 20 loss 0.7073414325714111
Epoch: 30 loss 0.47033336758613586
Epoch: 40 loss 0.28592681884765625
Epoch: 50 loss 0.1440829187631607
Epoch: 60 loss 0.061210326850414276
Epoch: 70 loss 0.027866192162036896
Epoch: 80 loss 0.015180563554167747
Epoch: 90 loss 0.009592454880475998
Total Time: 1.750007152557373


In [25]:
correct = 0
with torch.no_grad():
    for i, data in enumerate(X_test):
        y_val = gpumodel.forward(data)
        print(f'{i+1:2}. {str(y_val):38} {y_test[i]}')
        if y_val.argmax().item() == y_test[i]:
            correct += 1

print(f'\n{correct} out of {len(y_test)} = {100 * correct/len(y_test):.2f}% correct')

 1. tensor([-1.6505,  4.5929, -0.2764], device='cuda:0') 1
 2. tensor([-1.6505,  4.5929, -0.2764], device='cuda:0') 1
 3. tensor([ 6.0232,  0.9371, -6.8948], device='cuda:0') 0
 4. tensor([-1.6505,  4.5929, -0.2764], device='cuda:0') 1
 5. tensor([-12.3797,   4.9262,  10.3084], device='cuda:0') 2
 6. tensor([-12.3797,   4.9262,  10.3084], device='cuda:0') 2
 7. tensor([ 6.0232,  0.9371, -6.8948], device='cuda:0') 0
 8. tensor([ 6.0232,  0.9371, -6.8948], device='cuda:0') 0
 9. tensor([-12.3797,   4.9262,  10.3084], device='cuda:0') 2
10. tensor([-12.3797,   4.9262,  10.3084], device='cuda:0') 2
11. tensor([-12.3797,   4.9262,  10.3084], device='cuda:0') 2
12. tensor([ 6.0232,  0.9371, -6.8948], device='cuda:0') 0
13. tensor([-12.3797,   4.9262,  10.3084], device='cuda:0') 2
14. tensor([-1.6505,  4.5929, -0.2764], device='cuda:0') 1
15. tensor([-12.3797,   4.9262,  10.3084], device='cuda:0') 2
16. tensor([-1.6505,  4.5929, -0.2764], device='cuda:0') 1
17. tensor([-12.3797,   4.9262,  10